## Okay, Apparently I was crazy enough to try to write Java inside a Jupyter Notebook.

It's a wordle solver, AND IT WORKS. Pretty cool, fixed the code 3 years later.

In [3]:
!apt update -qq
!apt-get install -qq -y openjdk-11-jdk-headless
!curl -L https://github.com/SpencerPark/IJava/releases/download/v1.3.0/ijava-1.3.0.zip -o ijava-kernel.zip
!unzip -q ijava-kernel.zip -d ijava-kernel && cd ijava-kernel && python3 install.py --sys-prefix
!jupyter kernelspec list

38 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 3287k  100 3287k    0     0  5466k      0 --:--:-- --:--:-- --:--:-- 5466k
/content/ijava-kernel/install.py:164: DeprecationWarning: replace is ignored. Installing a kernelspec always replaces an existing installation
  install_dest = KernelSpecManager().install_kernel_spec(
Installed java kernel into "/usr/share/jupyter/kernels/java"
0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s

In [4]:
!javac -version || echo "javac not found"
!java -version

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
javac 11.0.28
openjdk version "11.0.28" 2025-07-15
OpenJDK Runtime Environment (build 11.0.28+6-post-Ubuntu-1ubuntu122.04.1)
OpenJDK 64-Bit Server VM (build 11.0.28+6-post-Ubuntu-1ubuntu122.04.1, mixed mode, sharing)


In [11]:
%%bash
set -e

cat > WordleSolverApp.java <<'JAVA'
import java.util.*;
import java.util.concurrent.ThreadLocalRandom;

class WordBank {
    static final String[] WORDS = {
        "cigar","rebut","sissy","humph","awake","blush","focal","evade","naval","serve",
        "heath","dwarf","model","karma","stink","grade","quiet","bench","abate","feign",
        "major","death","fresh","crust","stool","colon","abase","marry","react","batty",
        "pride","floss","helix","croak","stave","paint","wrung","vapid","plaid","brine",
        "clown","widow","irony","ruder","cater","robin","ultra","canny","stout","panel",
        "prose","gleam","torch","niche","quest","vigor","bison","latch","mourn","zesty",
        "adobe","prism","chore","fling","tiger","spear","quail","oxide","nymph","fuzzy",
        "grace","sound","tilde","mango","piano","trace","slate","irate","soare","crate",
        "spore","trend","glory","shank","knead","petal","bound","merit","cubic","apron",
        "sugar","ocean","elite","dodge","happy","knoll","eerie","robot","belle","pixie"
    };
    static void print10PerLine() {
        for (int i = 0; i < WORDS.length; i++) {
            System.out.print(WORDS[i] + (i % 10 == 9 ? "\n" : (i == WORDS.length-1 ? "\n" : ", ")));
        }
    }
}

class WordleCore {
    static int pattern(String guess, String target) {
        int[] marks = new int[5];
        int[] count = new int[26];
        for (int i = 0; i < 5; i++) count[target.charAt(i) - 'a']++;
        // Greens
        for (int i = 0; i < 5; i++) {
            if (guess.charAt(i) == target.charAt(i)) {
                marks[i] = 2;
                count[guess.charAt(i) - 'a']--;
            }
        }
        // Yellows
        for (int i = 0; i < 5; i++) {
            if (marks[i] == 0) {
                int idx = guess.charAt(i) - 'a';
                if (count[idx] > 0) { marks[i] = 1; count[idx]--; }
            }
        }
        int pat = 0;
        for (int i = 0; i < 5; i++) pat = pat * 3 + marks[i];
        return pat;
    }
    static String pretty(int pattern) {
        int[] m = new int[5];
        for (int i = 4; i >= 0; i--) { m[i] = pattern % 3; pattern /= 3; }
        StringBuilder sb = new StringBuilder();
        for (int x : m) sb.append(x==2 ? "🟩" : x==1 ? "🟨" : "⬛");
        return sb.toString();
    }
}

class Precompute {
    final String[] words;
    final int n;
    final Map<String,Integer> index = new HashMap<>();
    final int[][] pat; // pat[g][t] = feedback pattern for guess g vs target t
    Precompute(String[] words) {
        this.words = words; this.n = words.length;
        for (int i = 0; i < n; i++) index.put(words[i], i);
        pat = new int[n][n];
        for (int g = 0; g < n; g++)
            for (int t = 0; t < n; t++)
                pat[g][t] = WordleCore.pattern(words[g], words[t]);
    }
}

class Scorer {
    static class Score implements Comparable<Score> {
        final double expected;  // smaller better
        final int worst;        // smaller better
        final double entropy;   // larger better
        final boolean inCandidates; // prefer true
        final int guessIdx;
        Score(double exp, int worst, double ent, boolean inCand, int g) {
            this.expected=exp; this.worst=worst; this.entropy=ent; this.inCandidates=inCand; this.guessIdx=g;
        }
        public int compareTo(Score o) {
            int c;
            c = Double.compare(this.expected, o.expected); if (c!=0) return c;
            c = Integer.compare(this.worst, o.worst);      if (c!=0) return c;
            c = -Double.compare(this.entropy, o.entropy);  if (c!=0) return c;
            if (this.inCandidates != o.inCandidates) return this.inCandidates ? -1 : 1;
            return Integer.compare(this.guessIdx, o.guessIdx);
        }
    }

    static Score evaluate(Precompute P, int guess, List<Integer> cand, boolean[] isCand) {
        int[] counts = new int[243]; // 3^5 patterns
        for (int t : cand) counts[P.pat[guess][t]]++;
        int worst = 0; double sumSq = 0.0, H = 0.0, N = cand.size();
        for (int c : counts) if (c>0) {
            worst = Math.max(worst, c);
            sumSq += (double)c * c;
            double p = c / N;
            H += -p * (Math.log(p)/Math.log(2));
        }
        double expected = sumSq / N;
        return new Score(expected, worst, H, isCand[guess], guess);
    }
}

class Solver {
    final Precompute P;
    Solver(Precompute P){ this.P = P; }

    List<String> solve(String secret, boolean verbose){
        Integer secIdx = P.index.get(secret);
        if (secIdx == null) throw new IllegalArgumentException("Secret not in word bank.");

        List<Integer> cand = new ArrayList<>();
        boolean[] isCand = new boolean[P.n];
        for (int i = 0; i < P.n; i++){ cand.add(i); isCand[i]=true; }

        List<String> path = new ArrayList<>();
        for (int turn = 1; turn <= 6; turn++) {
            // Choose best guess across the whole bank (bank == guess set == answer set)
            Scorer.Score best = null;
            for (int g = 0; g < P.n; g++) {
                Scorer.Score s = Scorer.evaluate(P, g, cand, isCand);
                if (best == null || s.compareTo(best) < 0) best = s;
            }
            int guessIdx = best.guessIdx;
            String guess = P.words[guessIdx];
            path.add(guess);

            int pat = P.pat[guessIdx][secIdx];
            if (verbose) System.out.printf("%d. %s  %s%n", turn, guess, WordleCore.pretty(pat));
            if (guessIdx == secIdx) break;
            List<Integer> next = new ArrayList<>();
            Arrays.fill(isCand, false);
            for (int t : cand) {
                if (P.pat[guessIdx][t] == pat){ next.add(t); isCand[t]=true; }
            }
            cand = next;
            if (cand.isEmpty()) { if (verbose) System.out.println("No candidates remain."); break; }
        }
        if (verbose) {
            if (path.get(path.size()-1).equals(secret))
                System.out.printf("Solved in %d guesses.%n", path.size());
            else
                System.out.println("Failed within 6 guesses.");
        }
        return path;
    }

    void demo() {
        System.out.println("Using unified dictionary of " + P.n + " words (10 per line):");
        WordBank.print10PerLine();
        System.out.println();

        String pick = WordBank.WORDS[ThreadLocalRandom.current().nextInt(P.n)];
        System.out.println("Demo — secret: " + pick);
        solve(pick, true);

        // Small benchmark over 20 random secrets
        int trials = 20, solved=0, sum=0, worst=0;
        for (int i=0;i<trials;i++){
            String s = WordBank.WORDS[ThreadLocalRandom.current().nextInt(P.n)];
            List<String> path = solve(s, false);
            boolean ok = path.get(path.size()-1).equals(s);
            if (ok){ solved++; sum += path.size(); worst = Math.max(worst, path.size()); }
        }
        if (solved>0) System.out.printf("Solved %d/%d. Avg guesses: %.2f. Worst: %d%n",
                solved, trials, (double)sum/solved, worst);
    }
}

public class WordleSolverApp {
    public static void main(String[] args) {
        Precompute P = new Precompute(WordBank.WORDS);
        Solver solver = new Solver(P);
        if (args.length > 0) solver.solve(args[0].toLowerCase(Locale.ROOT), true);
        else solver.demo();
    }
}
JAVA

javac WordleSolverApp.java
java WordleSolverApp


Using unified dictionary of 100 words (10 per line):
cigar, rebut, sissy, humph, awake, blush, focal, evade, naval, serve
heath, dwarf, model, karma, stink, grade, quiet, bench, abate, feign
major, death, fresh, crust, stool, colon, abase, marry, react, batty
pride, floss, helix, croak, stave, paint, wrung, vapid, plaid, brine
clown, widow, irony, ruder, cater, robin, ultra, canny, stout, panel
prose, gleam, torch, niche, quest, vigor, bison, latch, mourn, zesty
adobe, prism, chore, fling, tiger, spear, quail, oxide, nymph, fuzzy
grace, sound, tilde, mango, piano, trace, slate, irate, soare, crate
spore, trend, glory, shank, knead, petal, bound, merit, cubic, apron
sugar, ocean, elite, dodge, happy, knoll, eerie, robot, belle, pixie

Demo — secret: crate
1. soare  ⬛⬛🟩🟨🟩
2. grace  ⬛🟩🟩🟨🟩
3. crate  🟩🟩🟩🟩🟩
Solved in 3 guesses.
Solved 20/20. Avg guesses: 2.40. Worst: 3
